## Display ABF Data as a Line Graph using Plotly

### The data from the ABF Library is a Numpy Series.

In [ ]:
import os
import plotly.graph_objects as go
import pyabf
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema

VALID_FILENAME_CHARS = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

def get_abf_data(filename):
    abf = pyabf.ABF('/app/data/' + filename + '.abf')
    return pd.DataFrame(abf.sweepY, columns = ["Y"], index = abf.sweepX)


def plot_abf(fname, intervals, comparison_range=10, image_width=1500, image_height=500):
    abf_df = get_abf_data(fname)
    
    data_dir = '/app/data/' + fname
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    saved_image_layout = go.Layout(
        autosize=False,
        width=image_width,
        height=image_height
    )
    
    for interval in intervals:
        output_filename = fname + '-' + \
                          ''.join(c for c in str(interval[0]) if c in VALID_FILENAME_CHARS) + '-' + \
                          ''.join(c for c in str(interval[1]) if c in VALID_FILENAME_CHARS)
        
        # Get data for specific interval
        data = abf_df[interval[0]:interval[1]]
        
        # Calculate local minimums & maximums.
        data_min = data.iloc[argrelextrema(data['Y'].values, np.less_equal, order=comparison_range)[0]]['Y']
        data_max = data.iloc[argrelextrema(data['Y'].values, np.greater_equal, order=comparison_range)[0]]['Y']
        
        # Save the data to CSV
        data_min.to_csv(data_dir + '/' + output_filename + "-min.csv")
        data_max.to_csv(data_dir + '/' + output_filename + "-max.csv")
        data_min.index.values.tofile(data_dir + '/' + output_filename + "-min-time.txt", sep="\n")
        data_max.index.values.tofile(data_dir + '/' + output_filename + "-max-time.txt", sep="\n")
        
        # Create the chart for saving to file
        fig = go.Figure(layout=saved_image_layout)
        fig.add_trace(go.Scatter(x=data.index.values, y=(data['Y']).values.flatten(),
                      mode='lines', name=fname))        
        fig.add_trace(go.Scatter(x=data_min.index.values, y=data_min.values.flatten(),
                      mode='markers', name='Min'))
        fig.add_trace(go.Scatter(x=data_max.index.values, y=data_max.values.flatten(),
                      mode='markers', name='Max'))
        fig.update_layout(title='Volumetric Chart', xaxis_title='Seconds since experiment started', yaxis_title='Volume')
        fig.write_image(data_dir + '/' + output_filename + ".png")
        
        # Create chart for viewing in Jupyter
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data.index.values, y=(data['Y']).values.flatten(),
                      mode='lines', name=fname))        
        fig.add_trace(go.Scatter(x=data_min.index.values, y=data_min.values.flatten(),
                      mode='markers', name='Min'))
        fig.add_trace(go.Scatter(x=data_max.index.values, y=data_max.values.flatten(),
                      mode='markers', name='Max'))
        fig.update_layout(title='Volumetric Chart', xaxis_title='Seconds since experiment started', yaxis_title='Volume')
        fig.show()

        
'''
The following options can be changed, but you may not need to change for every chart

comparison_range_for_min_max - How many points on each side to use for the comparison to determine min / max point
saved_image_width - The width of the image saved to file
saved_image_height - The height of the image saved to file
'''

comparison_range_for_min_max = 75
saved_image_width = 1500
saved_image_height = 500

'''
The following need to be updated:
    filename - The file name being analyzed. It is assumed that the file already exists in the 'data' folder.
               The file name extension is not needed. It is assumed to be '.abf' file.
               
    test_intervals - The test intervals for which the data and charts need to be captured.
'''
filename = "07101901"
test_intervals = [ 
                   [200, 230], \
                   [400, 430], \
                   [560, 590], \
                   [1030, 1060], \
                   [1235, 1265], \
                   [1512, 1542], \
                   [1827, 1857], \
                   [1947, 1977], \
                   [2323, 2353], \
                 ]


plot_abf(filename, test_intervals, \
         comparison_range=comparison_range_for_min_max, \
         image_width=saved_image_width, \
         image_height=saved_image_height)